# Discord Embed

## Imports

In [1]:
import json
import discord
from fuzzywuzzy.process import extractOne

### Read-in

In [2]:
with open("../Data/cleaned_characters.json", encoding="UTF-8") as f:
    data = json.load(f)

In [3]:
operators = list(filter(lambda x: x not in ['Medic Drone', 'Tentacle', 'Mirage', 'Roadblock', 'Stun Generator', 'Gate', 'Detector', 'Jammer', 'Turret'], data.keys()))

In [4]:
cyrillic = {
    "Зима" : "Zima",
    "Гум" : "Gummy",
    "Истина" : "Istina"
}

## Transform

In [5]:
def make_fields(unit):
    fields = [{"name" : "Trait", "value" : unit["trait"]}] + make_talents(unit) + make_skills(unit)
    return fields

In [6]:
def make_talents(unit):
    return [{"name" : key, "value" : value} for talent in unit["talents"] for key, value in talent.items()]

In [7]:
def make_skills(unit):
    return [{"name" : key, "value" : value, "inline" : True} for skill in unit["skills"] for key, value in skill.items()]

In [8]:
def build_dict(name, elite=None):
    unit = data[name]
    url = "https://aceship.github.io/AN-EN-Tags/akhrchars.html?opname="
    icon_url = "https://raw.githubusercontent.com/Aceship/AN-EN-Tags/master/img/classes/class_{}.png"
    av_url = "https://raw.githubusercontent.com/Aceship/AN-EN-Tags/master/img/avatars/{}.png"
    embed_dict = {
        "title" : name,
        "description" : "★" * unit["rarity"],
        "url" : url + name,
        "thumbnail" : {
            "url" : av_url.format(unit["internal_id"])
        },
        "image" : {
            "url" : unit["image"]
        },
        "author" : {
            "name" : unit["class"],
            "icon_url" : icon_url.format(unit["class"].lower())
        },
        "fields" : make_fields(unit)
    }
    
    if elite:
        #congratulations to amiya for having to do this for her E1 art
        if name == "Amiya" and elite == 1:
            embed_dict["image"]["url"] = embed_dict["image"]["url"][:-4] + "+.png"
            embed_dict["thumbnail"]["url"] = embed_dict["thumbnail"]["url"][:-4] + "_1+.png"
        elif unit["rarity"] >= 4:
            embed_dict["image"]["url"] = embed_dict["image"]["url"][:-5] + str(elite) + embed_dict["image"]["url"][-4:]
            embed_dict["thumbnail"]["url"] = embed_dict["thumbnail"]["url"][:-4] + "_" + str(elite) + embed_dict["thumbnail"]["url"][-4:]
            
    return embed_dict

In [9]:
def build_arknights_embed(name, elite=None):
    return discord.Embed.from_dict(build_dict(name, elite=elite))

In [10]:
def arknights(name, elite=None):
    if cyrillic.get(name):
        name = cyrillic[name]
    result = extractOne(name, operators)
    embed = build_arknights_embed(result[0], elite=elite)
    return embed